In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import openpyxl

folderPath = "data"
allFiles = glob.glob(os.path.join(folderPath, "*.xlsx"))

dataframes = []
for file in allFiles:
    df = pd.read_excel(file)
    df.columns = df.columns.str.strip()

    if 'Legacy Acct ID' not in df.columns:
        df['Legacy Acct ID'] = pd.NA

    dataframes.append(df)

data = pd.concat(dataframes,ignore_index=True)

print(data.head())




  Publication  AccoutID  Status Bill Method Dist ID Route ID Day pattern  \
0      MTM_KJ    502538  Active  Office Pay     NaN      NaN     SoTWTFS   
1      MTM_KJ    502618  Active  Office Pay     NaN      NaN     SoTWTFS   
2      MTM_KJ    502688  Active  Office Pay     NaN      NaN     SoTWTFS   
3      MTM_KJ    502692  Active  Office Pay     NaN      NaN     SoTWTFS   
4      MTM_KJ    502693  Active  Office Pay     NaN      NaN     SoTWTFS   

          City State   Zip Legacy Acct ID        Rate Code LastStartDate  \
0   Litchfield    ME  4350           9060  MATHER_7DAY_940    2018-04-01   
1  Weeks Mills    ME  4358           9377  MATHER_7DAY_970    2018-04-01   
2        China    ME  4358           9756  MATHER_7DAY_830    2023-05-27   
3  South China    ME  4358           9765  MATHER_7DAY_960    2018-04-01   
4  South China    ME  4358           9766  MATHER_7DAY_950    2023-01-24   

  OriginalStartDate  OccupantID RouteType ID  
0        2018-04-01     34591.0        

## Cleaning & Feature Engineering

## YEAR Month column for trends

## Map to Digital vs Print

In [5]:
def classifySubscription(row):
    billMethod = str(row['Bill Method']).upper()
    rateCode = str(row['Rate Code']).upper()

    if "DIGITAL" in billMethod or 'D' in rateCode:
        return "Digital"
    elif "PRINT" in billMethod or "P" in rateCode:
        return "Print"
    elif "BUNDLE" in billMethod or "B" in rateCode:
        return "Bundle"
    else:
        return "Other"

data['SubType'] = data.apply(classifySubscription,axis=1)

## Visualizations

In [6]:
current = data[data['Status'].str.upper() == 'ACTIVE'].copy()

subsByState = current.groupby('State')['AccoutID'].nunique().sort_values(ascending=False)

plt.figure(figsize=(10,10))
subsByState.head(20).plot(kind='bar')
plt.title('Active Subscriptions by State')
plt.xlabel('State')
plt.ylabel('Number of Accounts')
plt.tight_layout()
plt.show()


C:\Users\ydpra\AppData\Local\Temp\ipykernel_37704\2274122042.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## CITY

In [7]:
subByCity = current.groupby('City')['AccoutID'].nunique().sort_values(ascending=False)

plt.figure(figsize=(10,10))
subByCity.head(20).plot(kind='bar')
plt.title('Top 20 cities by Active Shareholders')
plt.xlabel('City')
plt.ylabel('Number of Accounts')
plt.tight_layout()
plt.show()

C:\Users\ydpra\AppData\Local\Temp\ipykernel_37704\2163789658.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Trends in subscribers over time

In [8]:
valid = data.dropna(subset=['LastStartDate']).copy()
valid['YearMonth'] = valid['LastStartDate'].dt.to_period('M')

subsOverTime = (
    valid.groupby(['YearMonth', 'AccoutID'])
    .size()
    .reset_index(name='num_accounts')
)

plt.figure(figsize=(10,10))
plt.plot(subsOverTime['YearMonth'].astype(str), subsOverTime['num_accounts'])
plt.xticks(rotation=45)
plt.title('Unique Subscribers over time')
plt.xlabel('Year-Month')
plt.ylabel('Number of Unique Accounts')
plt.tight_layout()
plt.show()

C:\Users\ydpra\AppData\Local\Temp\ipykernel_37704\1902835626.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


csacas

In [9]:
subsTimeType = (
    valid.groupby(['YearMonth', 'SubType'])['AccoutID']
    .nunique()
    .reset_index()
)

pivot = subsTimeType.pivot(index='YearMonth', columns='SubType', values='AccoutID').fillna(0)


plt.figure(figsize=(10,10))
for col in pivot.columns:
    plt.plot(pivot.index.astype(str), pivot[col], label=col)

plt.xticks(rotation=45)
plt.title('Subscribers over time by Subscription Type')
plt.xlabel('Year-Month')
plt.ylabel('Number of Unique Accounts')
plt.legend()
plt.tight_layout()
plt.show()

C:\Users\ydpra\AppData\Local\Temp\ipykernel_37704\2221167973.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="Subscription Dashboard", layout="wide")

st.title("📊 Subscription Analytics Dashboard")

# ----------------------------
# Upload CSV
# ----------------------------
uploaded_file = st.file_uploader("Upload your dataset", type=["csv"])

if uploaded_file:
    data = pd.read_csv(uploaded_file, parse_dates=["LastStartDate"])

    # Sidebar Filters
    st.sidebar.header("Filters")

    # STATUS FILTER
    status_filter = st.sidebar.multiselect(
        "Select Status",
        options=data["Status"].str.upper().unique(),
        default=["ACTIVE"]
    )

    # Filter data
    current = data[data["Status"].str.upper().isin(status_filter)].copy()

    st.subheader("Active Subscriptions by State")
    subsByState = current.groupby("State")["AccoutID"].nunique().sort_values(ascending=False)

    fig1 = px.bar(
        subsByState.head(20),
        x=subsByState.head(20).index,
        y=subsByState.head(20).values,
        labels={"x": "State", "y": "Number of Accounts"}
    )
    st.plotly_chart(fig1, use_container_width=True)

    # ---------------------------------------------------------------
    st.subheader("Top 20 Cities by Active Shareholders")
    subByCity = current.groupby("City")["AccoutID"].nunique().sort_values(ascending=False)

    fig2 = px.bar(
        subByCity.head(20),
        x=subByCity.head(20).index,
        y=subByCity.head(20).values,
        labels={"x": "City", "y": "Number of Accounts"},
    )
    st.plotly_chart(fig2, use_container_width=True)

    # ---------------------------------------------------------------
    st.subheader("Unique Subscribers Over Time")

    valid = data.dropna(subset=["LastStartDate"]).copy()
    valid["YearMonth"] = valid["LastStartDate"].dt.to_period("M")

    subsOverTime = (
        valid.groupby(["YearMonth", "AccoutID"])
        .size()
        .reset_index(name="num_accounts")
    )

    fig3 = px.line(
        subsOverTime,
        x=subsOverTime["YearMonth"].astype(str),
        y="num_accounts",
        labels={"x": "Year-Month", "num_accounts": "Number of Unique Accounts"},
    )
    st.plotly_chart(fig3, use_container_width=True)

    # ---------------------------------------------------------------
    st.subheader("Subscribers Over Time by Subscription Type")

    subsTimeType = (
        valid.groupby(["YearMonth", "SubType"])["AccoutID"]
        .nunique()
        .reset_index()
    )
    pivot = subsTimeType.pivot(index="YearMonth", columns="SubType", values="AccoutID").fillna(0)

    fig4 = px.line(
        pivot,
        x=pivot.index.astype(str),
        y=pivot.columns,
        labels={"x": "Year-Month", "value": "Subscribers"},
    )
    st.plotly_chart(fig4, use_container_width=True)

else:
    st.info("Upload a CSV to begin.")


2025-11-24 11:40:07.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.032 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-24 11:40:07.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar